# Relative Strength Index

The Relative Strength Index (RSI) is a momentum indicator that describes the current price relative to average high and low prices over a previous trading period. This indicator estimates overbought or oversold status and helps spot trend reversals, price pullbacks, and the emergence of bullish or bearish markets.

<img src="rsi.jpeg" width=300 height=300 />


- https://www.alpharithms.com/relative-strength-index-rsi-in-python-470209/#:~:text=Use%20the%20RS%20value%20to,both%20the%20gains%20and%20losses
- https://www.tradingview.com/support/solutions/43000502338-relative-strength-index-rsi/

In [1]:
# import basiclibraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import altair as alt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
## Function to show dataframes side by side
## https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

from IPython.display import display_html
from itertools import chain,cycle
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [3]:
#three ways to import finance data, 
#https://pypi.org/project/yfinance/
#https://www.alpharithms.com/python-financial-data-491110/
import yfinance as yf

Ticker="AVAX-USD"
df = yf.Ticker(Ticker).history(period='6mo')

#valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

# import quandl
# df2 = quandl.get('WIKI/NVDA')

# import pandas_datareader as pdr
# #request API call from https://data.nasdaq.com/sign-up
# df3 = pdr.get_data_quandl("NVDA", api_key="vjsci8v8EfV-BpoQmxbR")

In [4]:
# df['Difference'] = df['Close'].diff()
# df['Difference'].fillna(0,inplace=True)
# df['PercentChange'] = round(df['Difference']/ df['Close']*100,2)
# df.loc[df['Difference']>0, 'Gain'] = df['Difference']
# df.loc[df['Difference']<0, 'Loss'] = -df['Difference']
# df['Gain'].fillna(0,inplace=True)
# df['Loss'].fillna(0,inplace=True)

In [5]:
# for i in range(0,df.shape[0]-14):
    
#     print("Iteration: ", i)
#     sumGain=0
#     sumLoss=0
#     for index, row in df[1+i:14+i].iterrows():
#         sumGain=sumGain+row['Gain']
#         sumLoss=sumLoss+row['Loss']
#     df.loc[df.index==(14+i),'AvgGain']=round(sumGain/8,2)
#     df.loc[df.index==(14+i),'SumGain']=sumGain    
#     df.loc[df.index==(14+i),'AvgLoss']=round(sumLoss/8,2)    
#     df.loc[df.index==(14+i),'SumLoss']=sumLoss    

        

In [6]:
# df['RS'] = round(df['AvgGain']/df['AvgLoss'],2)
# df['AvgGain'].fillna(0,inplace=True)
# df['AvgLoss'].fillna(0,inplace=True)
# df['RS'].fillna(0,inplace=True)
# df['RSI'] = round(100-(100/(1+df['RS'])),2)
# df.drop(range(0,14),inplace=True)

In [7]:
df=df.reset_index()
df['Date']=pd.to_datetime(df['Date'])

In [8]:
df=df[['Date','Open','High','Low','Close']]
df['Close']=round(df['Close'],2)

In [9]:
df['RSI']=round(ta.rsi(df['Close'], timeperiod=14),2)
df.drop(range(0,14),inplace=True)
df.reset_index(drop=True,inplace=True)

In [10]:
df['Oversold']=30
df['Overbought']=70

In [11]:
Base = alt.Chart(df).mark_line().encode(
    x='Date:T',
    y='RSI:Q'
).interactive()

Oversold = alt.Chart(df).mark_line(color= 'red').encode(
    x='Date:T',
    y='Oversold:Q'
)

Overbought = alt.Chart(df).mark_line(color= 'green').encode(
    x='Date:T',
    y='Overbought:Q'
)

RSI_Chart=Base+Oversold+Overbought

In [12]:
#initial values
startAmount=1000
startPrice=df.iloc[0]['Close']
shares=round(startAmount/startPrice,3)

RSI_Sell=90
RSI_Buy=61

Sell_Flag=0

In [13]:
#Final Dateframe with Initialize Info
Final = pd.DataFrame(columns = ['Date','Price','Shares', 'Amount','RSI','Buy/Sell','Profit/Loss','Profit/Loss %'])
Final = Final.append({'Date' : df.iloc[0]['Date'], 
              'Price' : startPrice,
              'Shares' : shares, 
              'Amount' : startAmount,
              'RSI': df.iloc[0]['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0},ignore_index=True)

Long_Term = Final
Final.head()

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %
0,2021-10-10,56.59,17.671,1000,26.67,Buy,0,0


In [14]:
print('Start Price:', '${:,.2f}'.format(startPrice))
print('Start Shares: ', shares)
print('RSI Sell Signal: >=', RSI_Sell)
print('RSI Buy Signal: <', RSI_Buy)

Start Price: $56.59
Start Shares:  17.671
RSI Sell Signal: >= 90
RSI Buy Signal: < 61


In [15]:
for index, row in df.iterrows():
    if index!=0 and row['RSI']>=RSI_Sell and Sell_Flag==0:
        Sell_Flag=1
        Sold_Amount=Final.iloc[-1]['Shares']*row['Close']
        Final = Final.append({
              'Date' : row['Date'], 
              'Price' : row['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': row['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss': round(Sold_Amount-Final.iloc[-1]['Amount'],2),
              'Profit/Loss %': round((Sold_Amount-Final.iloc[-1]['Amount'])/Final.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
    elif index!=0 and row['RSI']<RSI_Buy and Sell_Flag==1:
        Sell_Flag=0
        Final = Final.append({
              'Date' : row['Date'], 
              'Price' : row['Close'],
              'Shares' : Final.iloc[-1]['Amount']/row['Close'], 
              'Amount' : Final.iloc[-1]['Amount'],
              'RSI': row['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0                              
                            },ignore_index=True)

In [16]:
if Final.iloc[-1]['Buy/Sell']=='Buy':
    Sold_Amount=df.iloc[-1]['Close']*Final.iloc[-1]['Shares']
    
    Final = Final.append({
              'Date' : df.iloc[-1]['Date'], 
              'Price' : df.iloc[-1]['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': df.iloc[-1]['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss':Sold_Amount-Final.iloc[-1]['Amount'],
              'Profit/Loss %':round((Sold_Amount-Final.iloc[-1]['Amount'])/Final.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)

In [17]:
Long_Term = Final.iloc[[0, -1]]
Long_Term.reset_index(drop=True,inplace=True)
Long_Term.at[1, 'Amount'] = Long_Term.iloc[-1]['Price']*Long_Term.iloc[0]['Shares']
Long_Term.at[1, 'Buy/Sell'] = 'Sell'
Long_Term.at[1, 'Profit/Loss'] = Long_Term.iloc[-1]['Amount']-Long_Term.iloc[0]['Amount']
Long_Term.at[1, 'Profit/Loss %'] = round(Long_Term.at[1, 'Profit/Loss']/Long_Term.iloc[0]['Amount']*100,2)

In [18]:
StockView=alt.Chart(df).mark_line().encode(
    x='Date:T',
    y='Close:Q',
    tooltip=['Close']
).properties(
    width=900,
    height=500
)

Base2 = alt.Chart(Final).mark_circle(color='Blue',size=90).encode(
    x='Date:T',
    y='RSI:Q',
    color='Buy/Sell',
    tooltip=['Date','RSI']
).properties(
    width=900,
    height=300
)



StockView & (RSI_Chart+Base2)

#.interactive()


alt.VConcatChart(...)

In [19]:
RSI_Chart
display(Final,Long_Term, titles=['RSI Strategy ','Long Term Buy&Hold'])

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %
0,2021-10-10,56.59,17.671,1000,26.67,Buy,0,0
1,2022-03-26,86.40,0.000,1526.77,56.80,Sell,526.774,52.68
,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %
0,2021-10-10,56.59,17.671,1000,26.67,Buy,0,0
1,2022-03-26,86.40,0.000,1526.77,56.80,Sell,526.774,52.68
